In [20]:
import numpy as np
import pandas as pd
import glob
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 400)

In [21]:
file_prefix='done_bug_'
file_pattern = os.path.join("./data/jira_trem/", f'{file_prefix}*.csv')
csv_files = glob.glob(file_pattern)

dataframes = []
for file in csv_files:
    dataframes.append(pd.read_csv(file))
df = pd.concat(dataframes, ignore_index=True)
df.head()

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project lead id,Project description,Project url,Priority,Resolution,Assignee,Assignee Id,Reporter,Reporter Id,Creator,Creator Id,Created,Updated,Last Viewed,Resolved,Affects versions,Affects versions.1,Affects versions.2,Affects versions.3,Fix versions,Fix versions.1,Fix versions.2,Due date,Votes,Labels,Labels.1,Labels.2,Description,Environment,Watchers,Watchers.1,Watchers.2,Watchers.3,Watchers.4,Watchers.5,Watchers.6,Watchers.7,Watchers.8,Watchers Id,Watchers Id.1,Watchers Id.2,Watchers Id.3,Watchers Id.4,Watchers Id.5,Watchers Id.6,Watchers Id.7,Watchers Id.8,Log Work,Log Work.1,Log Work.2,Log Work.3,Log Work.4,Log Work.5,Log Work.6,Log Work.7,Log Work.8,Log Work.9,Original estimate,Remaining Estimate,Time Spent,Work Ratio,Σ Original Estimate,Σ Remaining Estimate,Σ Time Spent,Security Level,Inward issue link (Blocks),Outward issue link (Blocks),Inward issue link (Cloners),Outward issue link (Cloners),Inward issue link (Defect),Outward issue link (Defect),Inward issue link (Duplicate),Inward issue link (Polaris datapoint issue link),Inward issue link (Problem/Incident),Outward issue link (Problem/Incident),Inward issue link (Relates),Inward issue link (Relates).1,Inward issue link (Relates).2,Outward issue link (Relates),Outward issue link (Relates).1,Outward issue link (Relates).2,Attachment,Attachment.1,Attachment.2,Attachment.3,Attachment.4,Attachment.5,Attachment.6,Attachment.7,Attachment.8,Attachment.9,Attachment.10,Attachment.11,Attachment.12,Attachment.13,Attachment.14,Attachment.15,Attachment.16,Attachment.17,Attachment.18,Attachment.19,Attachment.20,Custom field (Actual end),Custom field (Actual start),Custom field (Affected hardware),Custom field (Affected services),Custom field (Approvals),Custom field (Backout plan),Custom field (Baseline end date),Custom field (Baseline start date),Custom field (Billable),Custom field (Category),Custom field (Change reason),Custom field (Change risk),Custom field (Change type),Custom field (Checklists),Custom field (Client),Custom field (Craft),Custom field (Date of First Response),Custom field (Date of first comment),Custom field (Dev Estimation (mds)),Custom field (Development),Custom field (End date),Custom field (Engagement Type),Custom field (Epic Color),Custom field (Epic Link),Epic Link Summary,Custom field (Epic Name),Custom field (Epic Status),Custom field (Estimation Zoom Level),Custom field (FirstBundle),Custom field (Flagged),Custom field (Goals),Custom field (Impact),Custom field (Implementation plan),Custom field (Investigation reason),Custom field (Issue Origin),Custom field (Issue color),Custom field (Locked forms),Custom field (Major incident),Custom field (Open forms),Custom field (Operational categorization),Custom field (Pending reason),Custom field (Percentage of allocation in the project),Custom field (Planned end),Custom field (Planned start),Custom field (Product categorization),Custom field (Program Increment),Custom field (Project overview key),Custom field (Project overview status),Custom field (QA Automation Estimation (mds)),Custom field (QA Manual Estimation (mds)),Custom field (Rank),Custom field (Reopen Counter),Custom field (Request Type),Custom field (Request language),Custom field (Request participants),Custom field (Responders),Custom field (Root cause),Satisfaction rating,Custom field (Satisfaction date),Custom field (Scope),Custom field (Sentiment),Custom field (Severity),Custom field (Severity).1,Custom field (Source),Sprint,Sprint.1,Sprint.2,Sprint.3,Sprint.4,Sprint.5,Sprint.6,Sprint.7,Sprint.8,Sprint.9,Sprint.10,Custom field (Start date),Custom field (Story Points),Custom field (Story point estimate),Custom field (Submitted forms),Custom field (TEST Long List),Custom field (TTFR),Custom field (TTR),Custom field (Target end),Custom field (Target start),Custom field (Task progress),Custom field (Team),Custom field (Test Environment),Custom field 

In [22]:
# 合并以 'Labels' 开头的字段为数组形式
label_columns = [col for col in df.columns if col.startswith('Labels')]
df['Label_List'] = df[label_columns].apply(lambda row: [x for x in row if pd.notna(x) and x != ''], axis=1)

comment_fields = [col for col in df.columns if col.startswith('Comment')]
df['Comments'] = df[comment_fields].apply(
    lambda row: '\n'.join(row.dropna().astype(str)) if row.notna().any() else '',
    axis=1
)

watcher_id_fields = [col for col in df.columns if col.startswith('Watchers Id')]
df['WatcherIds'] = df[watcher_id_fields].apply(
    lambda row: ','.join(row[row != 'unknown'].dropna().astype(str)) if row.notna().any() and (row != 'unknown').any() else '',
    axis=1
)

In [23]:
df.shape

(2630, 368)

In [24]:
cols = ['Summary', 'Issue id', 'Issue Type', 'Status', 'Priority', 'Resolution', 'Assignee Id', 'Reporter Id', 
            'Creator Id', 'Created', 'Resolved', 'Affects versions', 'Fix versions', 'Due date', 'Labels', 
            'Description', 'Environment', 'Original estimate', 'Time Spent', 'Security Level', 
            'Custom field (Affected services)',
            'Custom field (Billable)', 'Custom field (Category)', 'Custom field (Issue Origin)',
            'Custom field (Severity)', 'Sprint', 'Custom field (Start date)', 'Custom field (Test Environment)',
            'Parent', 'Status Category', 'Status Category Changed', 'Components', 'Label_List', 'Comments', 'WatcherIds']
df = df[cols]
df.head()

,Summary,Issue id,Issue Type,Status,Priority,Resolution,Assignee Id,Reporter Id,Creator Id,Created,Resolved,Affects versions,Fix versions,Due date,Labels,Description,Environment,Original estimate,Time Spent,Security Level,Custom field (Affected services),Custom field (Billable),Custom field (Category),Custom field (Issue Origin),Custom field (Severity),Sprint,Custom field (Start date),Custom field (Test Environment),Parent,Status Category,Status Category Changed,Components,Label_List,Comments,WatcherIds
0,when a user log out and other user log in the...,1157113,Bug,Done,Highest,Cannot Reproduce,5e1309decbf1830daa9ce746,5d796147458a170db4b07d69,5d796147458a170db4b07d69,11/Jul/24 3:19 PM,17/Jul/24 9:46 AM,1.4.5,1.4.6 [M],NaN,rbc,when a user is logged out and another user log...,NaN,NaN,NaN,NaN,NaN,No,NaN,Reported by Client post Release,Critical,Sprint 120,NaN,UAT,NaN,Done,17/Jul/24 9:46 AM,NaN,[rbc],11/Jul/24 3:57 PM;5d796147458a170db4b07d69;[~a...,"6319978eea661fd37d509328,5d796147458a170db4b07..."
1,Enterprise Users Unable to Login Due to New Lo...,1155756,Bug,Done,Highest,Cannot Reproduce,5e1309decbf1830daa9ce746,5ee212569c72bd0ac05bbd01,5ee212569c72bd0ac05bbd01,05/Jul/24 12:00 PM,17/Jul/24 9:45 AM,1.4.5,NaN,05/Jul/24 12:00 AM,rbc,h2. After the recent deployment of the new log...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Reported by Client post Release,Major,NaN,NaN,UAT,NaN,Done,17/Jul/24 9:45 AM,NaN,[rbc],05/Jul/24 12:01 PM;5e1309decbf1830daa9ce746;pa...,"6319978eea661fd37d509328,5d796147458a170db4b07..."
2,menus misaligned and unable to scan on PWB o...,1149973,Bug,Done,Highest,Won't Do,5e1309decbf1830daa9ce746,5d796147458a170db4b07d69,5d796147458a170db4b07d69,12/Jun/24 5:36 AM,16/Jul/24 11:45 AM,1.4.3.2,NaN,NaN,rbc,"Hello Team,\n\ncould you please take a look at...",NaN,NaN,7200.0,NaN,NaN,Yes,NaN,Reported by Client during feedback,Critical,NaN,NaN,Prod,NaN,Done,16/Jul/24 11:45 AM,NaN,[rbc],12/Jun/24 8:31 AM;5ee212569c72bd0ac05bbd01;[~a...,"6319978eea661fd37d509328,5d796147458a170db4b07..."
3,Unable to upload media in PWB,1148947,Bug,Done,Highest,Resolved,557058:1a1ce1d5-7204-4e82-8c91-054565eda5b4,5d796147458a170db4b07d69,5d796147458a170db4b07d69,08/Jun/24 5:11 AM,14/Jun/24 2:06 PM,1.4.3,1.4.5,NaN,rbc,when uploading media using comments or any oth...,NaN,21600.0,20700.0,NaN,NaN,NaN,NaN,Reported by Client during feedback,Critical,Sprint 117,11/Jun/24 3:40 PM,Preprod,NaN,Done,14/Jun/24 3:10 PM,NaN,[rbc],10/Jun/24 12:12 PM;5ee212569c72bd0ac05bbd01;Lu...,"6319978eea661fd37d509328,5d796147458a170db4b07..."
4,Field display (name and user id) inconsistent ...,1148338,Bug,Done,Highest,Resolved,712020:5e6872a2-61e9-4f04-9779-238a5e6ba048,5ee212569c72bd0ac05bbd01,5ee212569c72bd0ac05bbd01,06/Jun/24 5:47 AM,21/Jun/24 11:25 AM,1.4.3,1.4.5,NaN,NaN,"In sanm uat, We already changed the attribute ...",NaN,28800.0,28800.0,NaN,NaN,NaN,NaN,Reported by Client during feedback,Major,Sprint 118,20/Jun/24 8:11 AM,UAT,270169.0,Done,21/Jun/24 2:26 PM,NaN,[],18/Jun/24 2:38 PM;5ee212569c72bd0ac05bbd01;Lui...,"6319978eea661fd37d509328,5d796147458a170db4b07..."


In [25]:
# 重命名一些列名
df =df.rename(columns={
    'Custom field (Affected services)': 'Affected Services',
    'Custom field (Billable)': 'Billable',
    'Custom field (Category)': 'Category',
    'Custom field (Issue Origin)': 'Issue Origin',
    'Custom field (Severity)': 'Severity',
    'Custom field (Start date)': 'Start Date',
    'Custom field (Test Environment)': 'Test Environment'
})
df.dropna(subset=['Assignee Id'], inplace=True)
# 进行时间转换
df['Created'] = pd.to_datetime(df['Created'])
df['Resolved'] = pd.to_datetime(df['Resolved'])
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['Status Category Changed'] = pd.to_datetime(df['Status Category Changed'])
# 计算时间间隔，并转换为小时
df['Resolved Time'] = (df['Resolved'] - df['Created']).dt.total_seconds() / 60 / 60
df['Description'] = df['Description'].fillna('')
# 计算标题和详情的长度
df['Summary Length'] = df['Summary'].str.len()
df['Description Length'] = df['Description'].str.len()
# Parent 字段填空
df['Parent'] = df['Parent'].fillna(0)
df['Parent'] = df['Parent'].astype(int)
# Time Spent转换为小时
# df['Time Spent'] = df['Time Spent']/60/60
df['Issue Origin'] = df['Issue Origin'].fillna('Unknown')
df['Billable'] = df['Billable'].fillna('No')
df['Week'] = df['Created'].dt.isocalendar().week
df['Month'] = df['Created'].dt.month

df['Severity'] = df['Severity'].fillna('Unknown')
df['Labels'] = df['Labels'].fillna('Unknown')
df['Test Environment'] = df['Test Environment'].fillna('Unknown')
df['Affects versions'] = df['Affects versions'].fillna('Unknown')

In [26]:
# 删除值大部分为null的列
df.drop(columns=['Environment', 'Components', 'Category', 'Affected Services', 'Security Level', 'Due date'], inplace=True)

In [27]:
# 选取有用的列
data = df[['Issue id', 'Summary', 'Description', 'Comments', 'Assignee Id', 'Creator Id', 'Reporter Id', 'WatcherIds', 'Severity', 'Priority']]

In [28]:
data.drop_duplicates(inplace = True)
data.reset_index(drop = True, inplace = True)

In [29]:
data['Assignee Id'].value_counts()

5d9f12f745fad00dc1264b68    260
5e1309decbf1830daa9ce746    221
5d9311c20c2a5d0dd87aef8b    185
5e7b2350da12670c395bc88a    184
5aafb696d965062a79374e06    169
                           ... 
5d9b1b62ea65c10c3fdbac24      1
5c584b86adc28b6e6e96ea0b      1
5aec7046fad8eb2e0df72a64      1
611cedeb41ff95006a8f1fad      1
5d1b6835dea8360d16bd11f2      1
Name: Assignee Id, Length: 63, dtype: int64

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Issue id     2480 non-null   int64 
 1   Summary      2480 non-null   object
 2   Description  2480 non-null   object
 3   Comments     2480 non-null   object
 4   Assignee Id  2480 non-null   object
 5   Creator Id   2480 non-null   object
 6   Reporter Id  2480 non-null   object
 7   WatcherIds   2480 non-null   object
 8   Severity     2480 non-null   object
 9   Priority     2480 non-null   object
dtypes: int64(1), object(9)
memory usage: 193.9+ KB


In [31]:
data

,Issue id,Summary,Description,Comments,Assignee Id,Creator Id,Reporter Id,WatcherIds,Severity,Priority
0,1157113,when a user log out and other user log in the...,when a user is logged out and another user log...,11/Jul/24 3:57 PM;5d796147458a170db4b07d69;[~a...,5e1309decbf1830daa9ce746,5d796147458a170db4b07d69,5d796147458a170db4b07d69,"6319978eea661fd37d509328,5d796147458a170db4b07...",Critical,Highest
1,1155756,Enterprise Users Unable to Login Due to New Lo...,h2. After the recent deployment of the new log...,05/Jul/24 12:01 PM;5e1309decbf1830daa9ce746;pa...,5e1309decbf1830daa9ce746,5ee212569c72bd0ac05bbd01,5ee212569c72bd0ac05bbd01,"6319978eea661fd37d509328,5d796147458a170db4b07...",Major,Highest
2,1149973,menus misaligned and unable to scan on PWB o...,"Hello Team,\n\ncould you please take a look at...",12/Jun/24 8:31 AM;5ee212569c72bd0ac05bbd01;[~a...,5e1309decbf1830daa9ce746,5d796147458a170db4b07d69,5d796147458a170db4b07d69,"6319978eea661fd37d509328,5d796147458a170db4b07...",Critical,Highest
3,1148947,Unable to upload media in PWB,when uploading media using comments or any oth...,10/Jun/24 12:12 PM;5ee212569c72bd0ac05bbd01;Lu...,557058:1a1ce1d5-7204-4e82-8c91-054565eda5b4,5d796147458a170db4b07d69,5d796147458a170db4b07d69,"6319978eea661fd37d509328,5d796147458a170db4b07...",Critical,Highest
4,1148338,Field display (name and user id) inconsistent ...,"In sanm uat, We already changed the attribute ...",18/Jun/24 2:38 PM;5ee212569c72bd0ac05bbd01;Lui...,712020:5e6872a2-61e9-4f04-9779-238a5e6ba048,5ee212569c72bd0ac05bbd01,5ee212569c72bd0ac05bbd01,"6319978eea661fd37d509328,5d796147458a170db4b07...",Major,Highest
...,...,...,...,...,...,...,...,...,...,...
2475,108808,[Users table][API] The PUT request for creatin...,Request method:\tPUT\r\nRequest URI:\thttps://...,17/Jan/20 9:59 AM;5c865308909d7a681cb39a57;Iss...,5d6fa38234b3d20d97780a8f,5c865308909d7a681cb39a57,5c865308909d7a681cb39a57,5c865308909d7a681cb39a57,Critical,High
2476,107038,[FE][Users] After activating/deactivating a us...,Steps:\n1.Log in and browse to Users page\n2.S...,23/Jan/20 3:24 PM;5dadac52eb076d0c302b4408;Wor...,5d80903627bbf40c2c3278cd,5dadac52eb076d0c302b4408,5dadac52eb076d0c302b4408,5dadac52eb076d0c302b4408,Critical,High
2477,107024,[FE][Users] The inactivate user option from Us...,Actions:\n1.Log in and browse to Users page\n2...,27/Jan/20 3:08 PM;5c865308909d7a681cb39a57;Iss...,5d6fa38234b3d20d97780a8f,5dadac52eb076d0c302b4408,5dadac52eb076d0c302b4408,"5dadac52eb076d0c302b4408,5c865308909d7a681cb39a57",Critical,High
2478,106902,[UI][UX][Users table] The filter fields and ad...,Current implementation:\r\n !image-2020-01-09-...,27/Jan/20 9:01 AM;5aafb696d965062a79374e06;ple...,5aafb696d965062a79374e06,5c865308909d7a681cb39a57,5c865308909d7a681cb39a57,"5aafb696d965062a79374e06,5c865308909d7a681cb39a57",Critical,High


文本处理

In [32]:
import string, re, nltk
import spacy
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [33]:
# RegexpTokenizer
regexp = RegexpTokenizer("[\w']+")

# 转换为小写
def convert_to_lowercase(text):
    return text.lower()

# 去除文本两边空格
def strip_text(text):
    return text.strip()

# 移除标点符号
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "")
    return re.sub(f"[{re.escape(punct_str)}]", " ", text)

# 移除标题的标点符号
def remove_summary_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "")
    punct_str = punct_str.replace("_", "")
    return re.sub(f"[{re.escape(punct_str)}]", " ", text)

# 移除数字token
def remove_number_token(text):
    words = text.split()
    # 过滤掉纯数字的词
    filtered_words = [word for word in words if not re.match(r'^\d+$', word)]
    # 将词按空格合并成句子
    combined_sentence = ' '.join(filtered_words).strip()
    return combined_sentence

# 移除html标签
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(' ', text)

# 移除表情
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags = re.UNICODE)
    return emoji_pattern.sub(' ', text)

# 移除http链接
def remove_http(text):
    http = "https?://\S+|www\.\S+" # matching strings beginning with http (but not just "http")
    pattern = r"({})".format(http) # creating pattern
    return re.sub(pattern, " ", text)

# Dictionary of acronyms
acronyms_url = './data/english_acronyms.json'
acronyms_dict = pd.read_json(acronyms_url, typ = 'series')
acronyms_list = list(acronyms_dict.keys())

# remove html tags
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(' ', text)

# 移除文本中包含的image tag
def remove_image_tags(text):
    # Define the regular expression pattern to match the image tags
    pattern = re.compile(r'!.*?!')
    cleaned_text = pattern.sub(' ', text)
    return cleaned_text

# 移除文本中{}的内容
def remove_bracket(text):
    pattern = re.compile(r'\{.*?\}')
    return pattern.sub(' ', text)

# 移除文本中||的内容
def remove_table(text):
    pattern = re.compile(r'\|.*?\|')
    return pattern.sub(' ', text)

# 移除文本中**的内容
def remove_star(text):
    pattern = re.compile(r'\*.*?\*')
    return pattern.sub(' ', text)

# convert contractions in a text
def convert_acronyms(text):
    words = []
    for word in regexp.tokenize(text):
        if word in acronyms_list:
            words = words + acronyms_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted

# Dictionary of contractions
contractions_url = './data/english_contractions.json'
contractions_dict = pd.read_json(contractions_url, typ = 'series')
# List of contractions
contractions_list = list(contractions_dict.keys())

# convert contractions in a text
def convert_contractions(text):
    words = []
    for word in regexp.tokenize(text):
        if word in contractions_list:
            words = words + contractions_dict[word].split()
        else:
            words = words + word.split()
    
    text_converted = " ".join(words)
    return text_converted

# 移除停用词
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    
    # 使用正则表达式 tokenizer 处理缩写和标点
    tokenizer = RegexpTokenizer(r'\w+\'?\w+|\w+')
    words = tokenizer.tokenize(text)
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Stemming 词干提取，stemming. 如 "running", "runner" 会被转换成 "run".
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem

# Lemmatization 词形还原. 如Better被还原为good
spacy_lemmatizer = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])
#lemmatizer = WordNetLemmatizer()
def text_lemmatizer(text):
    text_spacy = " ".join([token.lemma_ for token in spacy_lemmatizer(text)])
    #text_wordnet = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text)]) # regexp.tokenize(text)
    return text_spacy
    #return text_wordnet

# 移除非字母的词
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

# 根据词性过滤单词, 如过滤连词(conjunctions), 介词(prepositions)。保留名词(nouns)、形容词和动词
def keep_pos(text):
    tokens = regexp.tokenize(text)
    tokens_tagged = nltk.pos_tag(tokens)
    #keep_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'FW']
    keep_tags = ['JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'FW', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    keep_words = [x[0] for x in tokens_tagged if x[1] in keep_tags]
    return " ".join(keep_words)

# Additional stopwords
alphabets = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
prepositions = ["about", "above", "across", "after", "against", "among", "around", "at", "before", "behind", "below", "beside", "between", "by", "down", "during", "for", "from", "in", "inside", "into", "near", "of", "off", "on", "out", "over", "through", "to", "toward", "under", "up", "with"]
prepositions_less_common = ["aboard", "along", "amid", "as", "beneath", "beyond", "but", "concerning", "considering", "despite", "except", "following", "like", "minus", "onto", "outside", "per", "plus", "regarding", "round", "since", "than", "till", "underneath", "unlike", "until", "upon", "versus", "via", "within", "without"]
coordinating_conjunctions = ["and", "but", "for", "nor", "or", "so", "and", "yet"]
correlative_conjunctions = ["both", "and", "either", "or", "neither", "nor", "not", "only", "but", "whether", "or"]
subordinating_conjunctions = ["after", "although", "as", "as if", "as long as", "as much as", "as soon as", "as though", "because", "before", "by the time", "even if", "even though", "if", "in order that", "in case", "in the event that", "lest", "now that", "once", "only", "only if", "provided that", "since", "so", "supposing", "that", "than", "though", "till", "unless", "until", "when", "whenever", "where", "whereas", "wherever", "whether or not", "while"]
others = ["ã", "å", "ì", "û", "ûªm", "ûó", "ûò", "ìñ", "ûªre", "ûªve", "ûª", "ûªs", "ûówe"]
custom_words = ["thank", "hi", "hello", "regard", "issue", "please", "cc"]
additional_stops = alphabets + prepositions + prepositions_less_common + coordinating_conjunctions + correlative_conjunctions + subordinating_conjunctions + others + custom_words

def remove_additional_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in additional_stops])

def clean_text(text):
    # 按换行符分割文本
    lines = text.split('\n')
    # 过滤以#开始的句子
    filtered_lines = [line for line in lines if not line.startswith(('#', '*'))]
    # 将句子按空格合并为一个句子
    combined_sentence = ' '.join(filtered_lines).strip()
    return combined_sentence

def clean_log(text):
    # 按换行符分割文本
    lines = text.split('\n')
    # 正则表达式匹配时间日期格式
    date_pattern = r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}'
    # 过滤以时间日期开头的句子
    filtered_lines = [line for line in lines if not re.match(date_pattern, line)]
    # 将句子按空格合并为一个句子
    combined_sentence = ' '.join(filtered_lines).strip()
    return combined_sentence

def remove_comment_header(text):
    pattern = re.compile(r'(\d{2}/[A-Za-z]{3}/\d{2} \d{1,2}:\d{2} [APM]+);([0-9a-fA-F:.-]+)')
    return pattern.sub(' ', text)


In [34]:
# 处理标题
def summary_normalizer(text):
    text = strip_text(text)
    text = convert_to_lowercase(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_number_token(text)
    text = discard_non_alpha(text)
    text = keep_pos(text)
    text = remove_additional_stopwords(text)
    text = strip_text(text)
    text = re.sub(' +', ' ', text)
    return text

# 处理内容
def description_normalizer(text):
    text = convert_to_lowercase(text)
    text = strip_text(text)
    text = re.sub('\n', ' ', text) # converting text to one line
    text = re.sub('\[.*?\]', ' ', text) # removing square brackets
    text = remove_http(text)
    text = remove_image_tags(text)
    text = remove_bracket(text)
    text = remove_table(text)
#    text = remove_star(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = convert_acronyms(text)
    text = convert_contractions(text)
    text = remove_stopwords(text)
    text = text_lemmatizer(text)
    text = discard_non_alpha(text)
    text = keep_pos(text)
    text = remove_additional_stopwords(text)
    text = re.sub(' +', ' ', text)  # replace multiple spaces with a single space
    text = strip_text(text)
    return text

# 处理内容
def comment_normalizer(text):
    text = remove_comment_header(text)
    text = convert_to_lowercase(text)
    text = strip_text(text)
    text = re.sub('\n', ' ', text) # converting text to one line
    text = re.sub('\[.*?\]', ' ', text) # removing square brackets
    text = remove_http(text)
    text = remove_image_tags(text)
    text = remove_bracket(text)
    text = remove_table(text)
#    text = remove_star(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = convert_acronyms(text)
    text = convert_contractions(text)
    text = remove_stopwords(text)
    text = text_lemmatizer(text)
    text = discard_non_alpha(text)
    text = keep_pos(text)
    text = remove_additional_stopwords(text)
    text = re.sub(' +', ' ', text)  # replace multiple spaces with a single space
    text = strip_text(text)
    return text

In [35]:
data['normalized_summary'] = data['Summary'].apply(summary_normalizer)
data['normalized_description'] = data['Description'].apply(description_normalizer)
#data['normalized_comment'] = data['Comments'].apply(comment_normalizer)
# Text列包含标题和描述以便后面进行一起处理
data['normalized_text'] = data['normalized_summary'] + ' ' + data['normalized_description']

In [36]:
#data[['normalized_comment','Comments']].to_csv('/home/ryan/Downloads/comments.csv', index=False, encoding='utf-8')

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from itertools import combinations
# from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
# from scipy.sparse import csr_matrix

In [38]:
# 处理用户数据的辅助函数
def process_users(row):
    users = set()
    users.update([row['Assignee Id'], row['Creator Id'], row['Reporter Id']])
    if pd.notna(row['WatcherIds']):
        users.update(row['WatcherIds'].split(','))
    return list(users)

# 生成词汇和用户的频繁项集
def generate_frequent_itemsets(data, min_support=0.01, ngram_range=(1, 3)):
    """
    生成频繁项集，包含词汇和用户组合。
    
    :param data: 包含 'Description' 和用户列的 DataFrame
    :param min_support: 最小支持度
    :param ngram_range: ngram 词汇组合的范围
    :return: frequent_words, frequent_user_combos (频繁词汇和用户组合)
    """
    # 1. 分词和生成ngram
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    X = vectorizer.fit_transform(data['normalized_text'])
    words = vectorizer.get_feature_names_out()

    # 2. 词汇频次统计
    word_frequency = {word: 0 for word in words}
    for row in X.toarray():
        for i, val in enumerate(row):
            if val > 0:
                word_frequency[words[i]] += 1

    # 3. 用户组合统计
    data['Users'] = data.apply(process_users, axis=1)
    user_combinations_frequency = {}
    
    for users in data['Users']:
        for size in range(1, len(users) + 1):
            for combo in combinations(users, size):
                combo = tuple(sorted(combo))  # 保证组合的顺序一致
                if combo in user_combinations_frequency:
                    user_combinations_frequency[combo] += 1
                else:
                    user_combinations_frequency[combo] = 1

    # 4. 过滤支持度低于阈值的项
    num_records = len(data)
    frequent_words = {word: freq for word, freq in word_frequency.items() if freq / num_records >= min_support}
    frequent_user_combos = {combo: freq for combo, freq in user_combinations_frequency.items() if freq / num_records >= min_support}
    
    return frequent_words, frequent_user_combos

In [39]:
# 生成关联规则
def generate_association_rules(frequent_words, frequent_user_combos, data, min_lift=None):
    """
    根据频繁词汇和用户组合生成关联规则，并可根据最小提升度进行过滤。
    
    :param frequent_words: 频繁词汇字典
    :param frequent_user_combos: 频繁用户组合字典
    :param data: 包含 'Description' 和 'Users' 的 DataFrame
    :param min_lift: 最小提升度（可选），若指定则过滤提升度低于该值的规则
    :return: DataFrame 包含关联规则 (词汇 -> 用户)
    """
    num_records = len(data)
    rules = []

    # 5. 生成词汇到用户组合的关联规则
    for word in frequent_words:
        for combo, combo_freq in frequent_user_combos.items():
            # 计算词汇与用户组合的共现频次
            co_occurrence = sum((word in desc and all(user in data.iloc[idx]['Users'] for user in combo))
                                for idx, desc in enumerate(data['Description']))
            
            # 计算支持度和置信度
            if co_occurrence > 0:
                support = co_occurrence / num_records
                confidence = co_occurrence / frequent_words[word]
                lift = confidence / (combo_freq / num_records)

                # 存储规则
                rules.append({
                    'word': word,
                    'user_combo': combo,
                    'support': support,
                    'confidence': confidence,
                    'lift': lift
                })

    # 转换为DataFrame
    rules_df = pd.DataFrame(rules)
    
    # 根据min_lift过滤规则
    if min_lift is not None:
        rules_df = rules_df[rules_df['lift'] >= min_lift]
    
    # 返回按lift排序的规则
    return rules_df.sort_values(by='lift', ascending=False)

In [40]:
# 生成频繁项集
frequent_words, frequent_user_combos = generate_frequent_itemsets(data, min_support=0.01)

In [41]:
# 生成关联规则
rules_df = generate_association_rules(frequent_words, frequent_user_combos, data, min_lift=1)
# 输出关联规则
print(rules_df)

                 word                                         user_combo  \
24017        platform                        (5e412b35fd328a0c9ceb17c3,)   
26503             red  (5f0ead1a5ee2c300237703b3, 60ba2d0b3aa60500688...   
26502             red                        (5f0ead1a5ee2c300237703b3,)   
26452             red  (5f0eabbe1c682d002932d449, 60ba2d0b3aa60500688...   
26440             red  (5e1309decbf1830daa9ce746, 60ba2d0b3aa60500688...   
...               ...                                                ...   
22767         overlap  (5d6cf07c4b5fce0db430be45, 5d9f12f745fad00dc12...   
3919            cache                        (5cebc401a6198e0f2a7e629c,)   
10266       edit site  (5c3de2efdae8a34f8209088e, 5dadac52eb076d0c302...   
32935           table                        (5cebc401a6198e0f2a7e629c,)   
12689  federated user                        (5cebc401a6198e0f2a7e629c,)   

        support  confidence       lift  
24017  0.012903    1.230769  95.384615  
26503